In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [32]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

import sqlite3
import seaborn as sns

In [58]:
import chart_studio
chart_studio.tools.set_credentials_file(username='withwithout', api_key='Tx6Y0xDJjBrNhPtoaQZG')

In [59]:
# import plotly.graph_objs as go
# import plotly.plotly as py
# from plotly.offline import iplot, init_notebook_mode
# import cufflinks
# cufflinks.go_offline(connected=True)
# init_notebook_mode(connected=True)

# import plotly
# plotly.offline.init_notebook_mode(connected=True)
# import plotly.offline as py

In [34]:
%run src/pipeline
D = FraudData('data/data.json')
df = D.get_enriched_df()


In [35]:
idx= df.set_index(['object_id']).previous_payouts.apply(pd.Series).stack().index
tmp = pd.DataFrame(df.set_index(['object_id']).previous_payouts.apply(pd.Series).stack().values.tolist(),index=idx).reset_index().drop('level_1',1)

In [36]:
tmp.head()

,object_id,address,amount,country,created,event,name,state,uid,zip_code
0,786878,1700 North Brown Road,34389.38,US,2010-10-01 01:10:09,786869,RUF,GA,5419068,30043
1,786878,1700 North Brown Road,764.93,US,2011-02-02 01:12:10,786878,RUF,GA,5419068,30043
2,786878,1700 North Brown Road,17405.26,US,2011-03-15 01:30:42,1181203,RUF,GA,5419068,30043
3,786878,1700 North Brown Road,2224.16,US,2011-03-15 01:30:42,1293213,RUF,GA,5419068,30043
4,786878,1700 North Brown Road,5370.58,US,2011-04-01 01:30:31,1181203,RUF,GA,5419068,30043


In [37]:
event_currency = df.groupby(['object_id','currency']).size().reset_index()
event_currency = event_currency[['object_id','currency']]
event_currency.head()

,object_id,currency
0,47146,GBP
1,48129,GBP
2,48130,GBP
3,104008,GBP
4,104011,GBP


In [38]:
tmp2 = tmp.merge(event_currency, how='left', left_on = 'event', right_on = 'object_id')
tmp2.head()

,object_id_x,address,amount,country,created,event,name,state,uid,zip_code,object_id_y,currency
0,786878,1700 North Brown Road,34389.38,US,2010-10-01 01:10:09,786869,RUF,GA,5419068,30043,NaN,NaN
1,786878,1700 North Brown Road,764.93,US,2011-02-02 01:12:10,786878,RUF,GA,5419068,30043,786878.0,USD
2,786878,1700 North Brown Road,17405.26,US,2011-03-15 01:30:42,1181203,RUF,GA,5419068,30043,NaN,NaN
3,786878,1700 North Brown Road,2224.16,US,2011-03-15 01:30:42,1293213,RUF,GA,5419068,30043,NaN,NaN
4,786878,1700 North Brown Road,5370.58,US,2011-04-01 01:30:31,1181203,RUF,GA,5419068,30043,NaN,NaN


In [39]:
tmp2['fraud'] = tmp2['event'].isin(df.query('label == 1')['object_id'].to_list())
tmp2.fraud.value_counts()

# 1821 out of 1.09 million payouts were made to fraudulent events.

False    1095651
True        1821
Name: fraud, dtype: int64

In [40]:
tmp2.head()

,object_id_x,address,amount,country,created,event,name,state,uid,zip_code,object_id_y,currency,fraud
0,786878,1700 North Brown Road,34389.38,US,2010-10-01 01:10:09,786869,RUF,GA,5419068,30043,NaN,NaN,False
1,786878,1700 North Brown Road,764.93,US,2011-02-02 01:12:10,786878,RUF,GA,5419068,30043,786878.0,USD,False
2,786878,1700 North Brown Road,17405.26,US,2011-03-15 01:30:42,1181203,RUF,GA,5419068,30043,NaN,NaN,False
3,786878,1700 North Brown Road,2224.16,US,2011-03-15 01:30:42,1293213,RUF,GA,5419068,30043,NaN,NaN,False
4,786878,1700 North Brown Road,5370.58,US,2011-04-01 01:30:31,1181203,RUF,GA,5419068,30043,NaN,NaN,False


In [41]:
# We assume that the payout currency is the same as the currenty set for a given event.

# historical exchange rates
# https://www.ofx.com/en-us/forex-news/historical-exchange-rates/monthly-average-rates/

ex_rates = {'USD':1, 'GBP':1.5, 'CAD': 1,'AUD': 1, 'EUR': 1.3, 'NZD': 0.8}
ex_rates = pd.DataFrame.from_dict(ex_rates,orient='index')
ex_rates.reset_index()
ex_rates.columns = ['ex_rates']
ex_rates = ex_rates.rename_axis('currency').reset_index()

,index,0
0,USD,1.0
1,GBP,1.5
2,CAD,1.0
3,AUD,1.0
4,EUR,1.3
5,NZD,0.8


In [42]:
conn = sqlite3.connect("../events.db")
ex_rates.to_sql('ex_rates', conn, if_exists='replace')
tmp2.to_sql('payouts', conn, if_exists='replace')

In [43]:
tmp3 = pd.read_sql_query(
    'select a.*, b.ex_rates \
        from payouts as a left join ex_rates as b  \
        on a.currency = b.currency;' ,conn)
tmp3.head()

,index,object_id_x,address,amount,country,created,event,name,state,uid,zip_code,object_id_y,currency,fraud,ex_rates
0,0,786878,1700 North Brown Road,34389.38,US,2010-10-01 01:10:09,786869,RUF,GA,5419068,30043,NaN,None,0,NaN
1,1,786878,1700 North Brown Road,764.93,US,2011-02-02 01:12:10,786878,RUF,GA,5419068,30043,786878.0,USD,0,1.0
2,2,786878,1700 North Brown Road,17405.26,US,2011-03-15 01:30:42,1181203,RUF,GA,5419068,30043,NaN,None,0,NaN
3,3,786878,1700 North Brown Road,2224.16,US,2011-03-15 01:30:42,1293213,RUF,GA,5419068,30043,NaN,None,0,NaN
4,4,786878,1700 North Brown Road,5370.58,US,2011-04-01 01:30:31,1181203,RUF,GA,5419068,30043,NaN,None,0,NaN


In [44]:
# When currency info could not be inferred, we defaulted to USD (low impact - exchange rates 
# for CAD and AUD were also roughly 1 in the time period

tmp3['currency'] = tmp3['currency'].fillna('USD')
tmp3.currency.value_counts()
tmp3['amt_USD'] = np.where(tmp3['ex_rates'].isnull(),tmp3['amount'], tmp3['amount'] * tmp3['ex_rates'])
tmp3.head()

USD    1082488
GBP       8253
CAD       4199
AUD       1822
EUR        562
NZD        148
Name: currency, dtype: int64

,index,object_id_x,address,amount,country,created,event,name,state,uid,zip_code,object_id_y,currency,fraud,ex_rates,amt_USD
0,0,786878,1700 North Brown Road,34389.38,US,2010-10-01 01:10:09,786869,RUF,GA,5419068,30043,NaN,USD,0,NaN,34389.38
1,1,786878,1700 North Brown Road,764.93,US,2011-02-02 01:12:10,786878,RUF,GA,5419068,30043,786878.0,USD,0,1.0,764.93
2,2,786878,1700 North Brown Road,17405.26,US,2011-03-15 01:30:42,1181203,RUF,GA,5419068,30043,NaN,USD,0,NaN,17405.26
3,3,786878,1700 North Brown Road,2224.16,US,2011-03-15 01:30:42,1293213,RUF,GA,5419068,30043,NaN,USD,0,NaN,2224.16
4,4,786878,1700 North Brown Road,5370.58,US,2011-04-01 01:30:31,1181203,RUF,GA,5419068,30043,NaN,USD,0,NaN,5370.58


In [45]:
tmp3 = tmp3.set_index(pd.DatetimeIndex(tmp['created']))
tmp3.head()

,index,object_id_x,address,amount,country,created,event,name,state,uid,zip_code,object_id_y,currency,fraud,ex_rates,amt_USD
created,,,,,,,,,,,,,,,,
2010-10-01 01:10:09,0,786878,1700 North Brown Road,34389.38,US,2010-10-01 01:10:09,786869,RUF,GA,5419068,30043,NaN,USD,0,NaN,34389.38
2011-02-02 01:12:10,1,786878,1700 North Brown Road,764.93,US,2011-02-02 01:12:10,786878,RUF,GA,5419068,30043,786878.0,USD,0,1.0,764.93
2011-03-15 01:30:42,2,786878,1700 North Brown Road,17405.26,US,2011-03-15 01:30:42,1181203,RUF,GA,5419068,30043,NaN,USD,0,NaN,17405.26
2011-03-15 01:30:42,3,786878,1700 North Brown Road,2224.16,US,2011-03-15 01:30:42,1293213,RUF,GA,5419068,30043,NaN,USD,0,NaN,2224.16
2011-04-01 01:30:31,4,786878,1700 North Brown Road,5370.58,US,2011-04-01 01:30:31,1181203,RUF,GA,5419068,30043,NaN,USD,0,NaN,5370.58


In [85]:
payouts_fraud = tmp3.query('fraud == 1')

In [83]:
payouts_fraud['name'].value_counts()

                                1754
Global Gas Card                   25
Steven Kadlec                     14
Jason Phillips                     8
PARTY STARZ ENT INC                8
C.Cornish                          6
robert carter                      2
Royal Princesses and Knights       2
Annie Zurth                        1
Midtown Bar & Restaurant LLC       1
Name: name, dtype: int64

In [84]:
import chart_studio.plotly as py
import plotly.graph_objects as go


fig = go.Figure(go.Scatter(x=payouts_fraud.created, y=payouts_fraud.amt_USD, text=payouts_fraud.state, mode='markers',name='State'))
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Payout Amounts $')

py.iplot(fig, filename='payout_amt_fraud')

In [86]:
payouts_fraud = payouts_fraud.amt_USD.resample('M').sum().reset_index()

In [87]:
payouts_fraud.to_csv('data/monthly_payouts_FraudOnly.csv')

In [88]:
payouts_all = tmp3.amt_USD.resample('M').sum().reset_index()

In [93]:
fig = go.Figure(go.Scatter(x=payouts_all.created, y=payouts_all.amt_USD, mode='markers'))
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Payout Amounts $')

py.iplot(fig, filename='payout_amt_all')

In [91]:
payouts_all.to_csv('data/monthly_payouts_AllEvents.csv')

In [76]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=payouts_fraud.created, y= payouts_fraud['amt_USD'], name="Payout Amount for Fraudulent Events",
#                          line_color='deepskyblue'))

# fig.add_trace(go.Scatter(x=payouts_all.created, y=payouts_all['amt_USD'], name="Payout Amout for All Events",
#                          line_color='dimgray'))

# fig.update(title='Time Series with Rangeslider', xaxis_rangeslider_visible=True)
# fig.show()

In [77]:
payouts_fraud = payouts_fraud.set_index('created')

payouts_fraud.iplot(y='amt_USD', mode = 'lines', xTitle = 'Date', yTitle = 'Payout Amount$',\
                    title = 'Payout Amount for Fraudulent Events: USD')

# py.offline.plot(data, filename='myplot.html')



AttributeError: 'DataFrame' object has no attribute 'iplot'

In [53]:
payouts_all = payouts_all.set_index('created')

payouts_all.iplot(y='amt_USD', mode = 'lines', xTitle = 'Date', yTitle = 'Payout Amount$',\
                    title = 'Payout Amount for All the Events: USD')

AttributeError: 'DataFrame' object has no attribute 'iplot'